
# CAPSTONE — Telco × Energy (EU/UK) 

In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


### Paths & ROI parameters

In [2]:

BASE = Path("/Users/bella/Downloads/vodafone/Methodology/capstone_outputs_final")
BASE.mkdir(parents=True, exist_ok=True)
OUT = BASE

# Optional input (press-facts) files if you already have them.
# If missing, the script auto-creates a minimal league so it still runs E2E.
P_VODA_SUEZ = OUT / "vodafone_suez_press_facts.csv"
P_ORANGE_BZ = OUT / "orange_veolia_birdz_press_facts.csv"

# Outputs
LEAGUE_OUT    = OUT / "eu_telco_energy_league_press_facts.csv"
RANKED_OUT    = OUT / "league_ranked_speed.csv"
VODA_PROJ_CSV = OUT / "vodafone_suez_projection_from_public_target.csv"
VODA_PROJ_PNG = OUT / "vodafone_suez_adoption.png"
VALUE_CURVE_PNG = OUT / "vodafone_suez_value_curve_MODEL.png"
SENS_TORNADO_PNG = OUT / "vodafone_suez_sensitivity_units_tornado.png"
MC_OUT        = OUT / "vodafone_suez_monte_carlo_summary.csv"
SENS_OUT      = OUT / "vodafone_suez_sensitivity_one_way.csv"
ROI_OUT       = OUT / "vodafone_suez_roi_summary.csv"
ROI_BAR_PNG   = OUT / "roi_comparison.png"
RADAR_PNG     = OUT / "kpi_radar.png"
WRITEUP_MD    = OUT / "capstone_results_writeup.md"
APPX_KPI_CSV  = OUT / "kpi_inputs_telco_energy.csv"
APPX_LEAGUE_MIN_CSV = OUT / "league_table_telco_energy_min.csv"
SECTOR_SMART_WATER_PNG = OUT / "plot_scale_smart_water.png"
TOP_COUNTRIES_CSV      = OUT / "top_countries_allocated_MODEL.csv"
TOP_COUNTRIES_PNG      = OUT / "top_countries_bar_MODEL.png"

# ROI model parameters (MODEL) — public-benchmark buckets
AVG_ANNUAL_WATER_SPEND_PER_CONNECTION_EUR = 300   # EU residential/SME magnitude
CONSUMPTION_REDUCTION_PCT = 0.15                  # “up to 15%” (Vodafone/SUEZ PR)
CAPEX_PER_METER_EUR = 100                         # meter+comms+install bucket
OPEX_PER_METER_PER_YEAR = 0.0                     # keep 0 if unknown
DISCOUNT_RATE_ANNUAL = 0.08                       # 8% WACC proxy

print("Observation: Output folder =", OUT)


Observation: Output folder = /Users/bella/Downloads/vodafone/Methodology/capstone_outputs_final



### Load (or auto-create) press-facts data

In [3]:
def load_if_exists(path: Path):
    if not path.exists():
        return None
    df = pd.read_csv(path)
    df.columns = df.columns.str.lower().str.strip()
    if "announcement_date" in df.columns:
        df["announcement_date"] = pd.to_datetime(df["announcement_date"], errors="coerce")
    for col in ("start_year","target_year","announced_units_or_capacity","claimed_savings_pct"):
        if col in df.columns: df[col] = pd.to_numeric(df[col], errors="coerce")
    for col in ("partnership","sector","countries","unit_type","notes","source_title","source_url"):
        if col in df.columns: df[col] = df[col].astype(str).str.strip()
    return df

dfs = []
for p in [P_VODA_SUEZ, P_ORANGE_BZ]:
    d = load_if_exists(p)
    if d is not None: dfs.append(d)

if not dfs:
    # Minimal auditable fallback from public press releases
    league_min = pd.DataFrame([
        {
            "partnership":"Vodafone–SUEZ","sector":"Smart water (NB‑IoT)",
            "countries":"UK; FR; IT; ES; NZ","announcement_date":"2024-04-15",
            "start_year":2024,"announced_units_or_capacity":2_000_000,
            "unit_type":"smart meters","target_year":2030,"claimed_savings_pct":15.0,
            "source_title":"SUEZ × Vodafone press release",
            "source_url":"https://www.suez.com/en/news/press-releases/suez-vodafone-sign-global-partnership-remote-reading-new-generation-smart-water-meters"
        },
        {
            "partnership":"Orange–Veolia/Birdz","sector":"Smart water (LoRaWAN)",
            "countries":"FR","announcement_date":"2018-12-19",
            "start_year":2018,"announced_units_or_capacity":3_000_000,
            "unit_type":"smart meters (connected)","target_year":2027,"claimed_savings_pct":np.nan,
            "source_title":"Orange newsroom — Birdz 3M meters (LoRa)",
            "source_url":"https://newsroom.orange.com/nova-veolia-and-its-subsidiary-birdz-choose-orange-business-services-to-help-them-digitalize-veolias-remote-water-meter-reading-services-in-france/"
        }
    ])
    league_min["announcement_date"] = pd.to_datetime(league_min["announcement_date"])
    dfs = [league_min]

league = pd.concat(dfs, ignore_index=True)
league.columns = [c.strip().lower().replace(" ","_") for c in league.columns]
print(f"Observation: merged league rows = {len(league)}")

# EU/UK focus (both already EU; left for completeness)
eu_countries = ["UK","United Kingdom","Italy","France","Germany","Spain","EU/Global","EU","Ireland","Portugal"]
league["eu_uk_flag"] = league["countries"].fillna("").apply(lambda s: any(c in s for c in eu_countries))
league_eu = league[league["eu_uk_flag"]].copy()
league_eu["label"] = league_eu["partnership"] + " — " + league_eu["sector"]
league.to_csv(LEAGUE_OUT, index=False)
league.to_csv(APPX_LEAGUE_MIN_CSV, index=False)
print(f"Observation: wrote {LEAGUE_OUT.name}")


Observation: merged league rows = 2
Observation: wrote eu_telco_energy_league_press_facts.csv



### Descriptive KPIs — pace & sector chart

In [4]:

def pace_units_per_year(row):
    if pd.notna(row.get("announced_units_or_capacity")) and pd.notna(row.get("start_year")) and pd.notna(row.get("target_year")):
        years = max(1, int(row["target_year"] - row["start_year"]))
        return row["announced_units_or_capacity"] / years
    return np.nan

league_eu["speed_index_units_per_year"] = league_eu.apply(pace_units_per_year, axis=1)
ranked = league_eu.sort_values("speed_index_units_per_year", ascending=False)
ranked.to_csv(RANKED_OUT, index=False)
print(f"Observation: wrote {RANKED_OUT.name}")

# Sector plot (smart water only)
sub = league_eu[(league_eu["sector"].str.contains("Smart water", case=False, na=False)) &
                (league_eu["announced_units_or_capacity"].notna())].copy()
if not sub.empty:
    sub = sub.sort_values("announced_units_or_capacity", ascending=True)
    plt.figure()
    plt.barh(sub["partnership"], sub["announced_units_or_capacity"])
    plt.title("Scale by partnership — Smart water")
    plt.xlabel("announced units")
    plt.tight_layout()
    plt.savefig(SECTOR_SMART_WATER_PNG, bbox_inches="tight"); plt.close()
    print(f"Observation: saved {SECTOR_SMART_WATER_PNG.name}")
else:
    print("Observation: no numeric smart-water values to chart; skipped.")


Observation: wrote league_ranked_speed.csv
Observation: saved plot_scale_smart_water.png


### Predictive modelling (MODEL) — adoption

In [5]:


voda_rows = league_eu[league_eu["partnership"].str.contains("Vodafone", case=False, na=False)]
K = int(voda_rows["announced_units_or_capacity"].dropna().max()) if not voda_rows.empty else 2_000_000
start_year = int(voda_rows["start_year"].dropna().min()) if not voda_rows.empty else 2024
target_year = int(voda_rows["target_year"].dropna().max()) if not voda_rows.empty else 2030
r = 0.55  # logistic steepness

years = np.arange(start_year, target_year + 1)
t = years - start_year
t0 = (target_year - start_year) / 2.0
proj_units = K / (1 + np.exp(-r*(t - t0)))
voda_proj = pd.DataFrame({"year": years, "projected_cumulative_units": proj_units})
voda_proj.to_csv(VODA_PROJ_CSV, index=False)
print(f"Observation (Model): wrote {VODA_PROJ_CSV.name}  [K={K:,}, start={start_year}, target={target_year}, r={r}]")

plt.figure()
plt.plot(voda_proj["year"], voda_proj["projected_cumulative_units"]/1e6, marker="o")
plt.title("Vodafone–SUEZ: projected cumulative units (MODEL)")
plt.xlabel("year"); plt.ylabel("units (millions)")
plt.grid(True); plt.tight_layout()
plt.savefig(VODA_PROJ_PNG, bbox_inches="tight"); plt.close()
print(f"Observation (Model): saved {VODA_PROJ_PNG.name}")

def milestone(df, frac=0.5):
    thr = K * frac
    sel = df[df["projected_cumulative_units"] >= thr]
    return None if sel.empty else int(sel.iloc[0]["year"])
m50 = milestone(voda_proj, 0.5)
m80 = milestone(voda_proj, 0.8)
print(f"Observation (Model): milestones — 50%: {m50}, 80%: {m80}")



Observation (Model): wrote vodafone_suez_projection_from_public_target.csv  [K=2,000,000, start=2024, target=2030, r=0.55]
Observation (Model): saved vodafone_suez_adoption.png
Observation (Model): milestones — 50%: 2027, 80%: 2030



### Value & ROI model (MODEL) — annual basis

In [6]:


annual_saving_per_meter = AVG_ANNUAL_WATER_SPEND_PER_CONNECTION_EUR * CONSUMPTION_REDUCTION_PCT
voda_val = voda_proj.copy()
voda_val["new_installs"] = voda_val["projected_cumulative_units"].diff().fillna(voda_val["projected_cumulative_units"])
voda_val["active_meters"] = voda_val["projected_cumulative_units"]
voda_val["yearly_savings_eur"] = voda_val["active_meters"] * annual_saving_per_meter
voda_val["yearly_capex_eur"] = voda_val["new_installs"] * CAPEX_PER_METER_EUR
voda_val["yearly_opex_eur"]  = voda_val["active_meters"] * OPEX_PER_METER_PER_YEAR
voda_val["net_cf_eur"] = voda_val["yearly_savings_eur"] - voda_val["yearly_capex_eur"] - voda_val["yearly_opex_eur"]
voda_val["cumulative_savings_eur"] = voda_val["yearly_savings_eur"].cumsum()
voda_val["cumulative_capex_eur"]   = voda_val["yearly_capex_eur"].cumsum()

# Payback year
cross = voda_val[voda_val["cumulative_savings_eur"] >= voda_val["cumulative_capex_eur"]]
payback_year = None if cross.empty else int(cross.iloc[0]["year"])

# NPV & IRR (annual)
def irr_bisect(cash, lo=-0.9, hi=1.0, tol=1e-7, it=200):
    def npv_rate(r):
        return (cash / ((1 + r) ** np.arange(len(cash)))).sum()
    if npv_rate(lo)*npv_rate(hi) > 0: return np.nan
    for _ in range(it):
        mid = (lo+hi)/2
        v = npv_rate(mid)
        if abs(v) < tol: return mid
        if npv_rate(lo)*v < 0: hi = mid
        else: lo = mid
    return mid

t = np.arange(len(voda_val))
disc = 1 / ((1 + DISCOUNT_RATE_ANNUAL) ** t)
npv_est = float((voda_val["net_cf_eur"].values * disc).sum())
irr_a = irr_bisect(voda_val["net_cf_eur"].values)

# ROI on horizon
capex_total = float(voda_val["cumulative_capex_eur"].iloc[-1])
savings_total = float(voda_val["cumulative_savings_eur"].iloc[-1])
roi_h = (savings_total - capex_total)/capex_total if capex_total>0 else np.nan

plt.figure()
plt.plot(voda_val["year"], voda_val["cumulative_savings_eur"]/1e6, label="Cumulative Savings")
plt.plot(voda_val["year"], voda_val["cumulative_capex_eur"]/1e6, label="Cumulative CAPEX")
plt.title("Vodafone–SUEZ: Value Curve (MODEL)")
plt.xlabel("year"); plt.ylabel("€ million"); plt.legend(); plt.tight_layout()
plt.savefig(VALUE_CURVE_PNG, bbox_inches="tight"); plt.close()
print(f"Observation (Model): saved {VALUE_CURVE_PNG.name}")

pd.DataFrame([{
    "annual_spend_per_connection_eur": AVG_ANNUAL_WATER_SPEND_PER_CONNECTION_EUR,
    "reduction_pct": CONSUMPTION_REDUCTION_PCT,
    "capex_per_meter_eur": CAPEX_PER_METER_EUR,
    "opex_per_meter_per_year": OPEX_PER_METER_PER_YEAR,
    "discount_rate_annual": DISCOUNT_RATE_ANNUAL,
    "payback_year": payback_year,
    "npv_eur": npv_est,
    "irr_annual": irr_a,
    "roi_horizon": roi_h,
    "milestone_50pct_year": m50,
    "milestone_80pct_year": m80,
    "K_target_units": K
}]).to_csv(ROI_OUT, index=False)
print(f"Observation (Model): wrote {ROI_OUT.name}")


Observation (Model): saved vodafone_suez_value_curve_MODEL.png
Observation (Model): wrote vodafone_suez_roi_summary.csv


### Sensitivity (one-way) + Monte Carlo

In [7]:


def value_curve_metrics(Ki, ri, start_y, annual_spend, red_pct, capex_pm, opex_py, disc_rate):
    yrs = np.arange(start_y, target_year+1)
    t_ = yrs - start_y
    t0_ = (target_year - start_y)/2.0
    y_ = Ki / (1 + np.exp(-ri*(t_ - t0_)))
    df_ = pd.DataFrame({"year": yrs, "projected_cumulative_units": y_})
    annual_sav = annual_spend * red_pct
    new_inst = df_["projected_cumulative_units"].diff().fillna(df_["projected_cumulative_units"])
    active   = df_["projected_cumulative_units"]
    yr_sav   = active * annual_sav
    yr_cap   = new_inst * capex_pm
    yr_opex  = active * opex_py
    net      = yr_sav - yr_cap - yr_opex
    cap_tot  = float(yr_cap.cumsum().iloc[-1])
    sav_tot  = float(yr_sav.cumsum().iloc[-1])
    roi      = (sav_tot - cap_tot)/cap_tot if cap_tot>0 else np.nan
    t        = np.arange(len(df_))
    disc     = 1/((1+disc_rate)**t)
    npv      = float((net.values * disc).sum())
    return float(y_[-1]), npv, roi

def one_way_sensitivity(K_base, r_base, start_base, spend, red, capex_pm, opex_py, disc):
    rows = []
    shocks = [
        ("K target (±10%)", "K", 0.10),
        ("Growth rate r (±10%)", "r", 0.10),
        ("Reduction pct (±20%)", "red", 0.20),
        ("Annual spend (±20%)", "spend", 0.20),
        ("CAPEX per meter (±20%)", "capex", 0.20),
        ("Start delay (±2 mo)", "delay_months", 2.0),
    ]
    base_units, base_npv, base_roi = value_curve_metrics(K_base, r_base, start_base, spend, red, capex_pm, opex_py, disc)
    for label, key, delta in shocks:
        if key == "K":
            u_plus, npv_plus, roi_plus = value_curve_metrics(K_base*(1+delta), r_base, start_base, spend, red, capex_pm, opex_py, disc)
            u_minus, npv_minus, roi_minus = value_curve_metrics(K_base*(1-delta), r_base, start_base, spend, red, capex_pm, opex_py, disc)
        elif key == "r":
            u_plus, npv_plus, roi_plus = value_curve_metrics(K_base, r_base*(1+delta), start_base, spend, red, capex_pm, opex_py, disc)
            u_minus, npv_minus, roi_minus = value_curve_metrics(K_base, r_base*(1-delta), start_base, spend, red, capex_pm, opex_py, disc)
        elif key == "red":
            u_plus, npv_plus, roi_plus = value_curve_metrics(K_base, r_base, start_base, spend, red*(1+delta), capex_pm, opex_py, disc)
            u_minus, npv_minus, roi_minus = value_curve_metrics(K_base, r_base, start_base, spend, red*(1-delta), capex_pm, opex_py, disc)
        elif key == "spend":
            u_plus, npv_plus, roi_plus = value_curve_metrics(K_base, r_base, start_base, spend*(1+delta), red, capex_pm, opex_py, disc)
            u_minus, npv_minus, roi_minus = value_curve_metrics(K_base, r_base, start_base, spend*(1-delta), red, capex_pm, opex_py, disc)
        elif key == "capex":
            u_plus, npv_plus, roi_plus = value_curve_metrics(K_base, r_base, start_base, spend, red, capex_pm*(1+delta), opex_py, disc)
            u_minus, npv_minus, roi_minus = value_curve_metrics(K_base, r_base, start_base, spend, red, capex_pm*(1-delta), opex_py, disc)
        else:
            u_plus, npv_plus, roi_plus = value_curve_metrics(K_base, r_base, start_base + (delta/12.0), spend, red, capex_pm, opex_py, disc)
            u_minus, npv_minus, roi_minus = value_curve_metrics(K_base, r_base, start_base - (delta/12.0), spend, red, capex_pm, opex_py, disc)
        rows.append({
            "parameter": label,
            "baseline_units_2030": base_units,
            "units_2030_minus": u_minus, "units_2030_plus": u_plus,
            "delta_units_minus": u_minus - base_units, "delta_units_plus": u_plus - base_units,
            "baseline_roi": base_roi, "roi_minus": roi_minus, "roi_plus": roi_plus,
            "baseline_npv": base_npv, "npv_minus": npv_minus, "npv_plus": npv_plus
        })
    return pd.DataFrame(rows)

sens = one_way_sensitivity(
    K, r, start_year,
    AVG_ANNUAL_WATER_SPEND_PER_CONNECTION_EUR,
    CONSUMPTION_REDUCTION_PCT,
    CAPEX_PER_METER_EUR,
    OPEX_PER_METER_PER_YEAR,
    DISCOUNT_RATE_ANNUAL
)
sens.to_csv(SENS_OUT, index=False)
print(f"Observation (Model): wrote {SENS_OUT.name}")

plt.figure()
labels = sens["parameter"]; neg = sens["delta_units_minus"]; pos = sens["delta_units_plus"]
y = np.arange(len(labels))
plt.barh(y, pos, left=0, label="+ shock"); plt.barh(y, neg, left=0, label="- shock")
plt.yticks(y, labels); plt.xlabel("Δ Units by 2030 (vs baseline)")
plt.title("Sensitivity — Units by 2030 (MODEL)"); plt.legend(); plt.tight_layout()
plt.savefig(SENS_TORNADO_PNG, bbox_inches="tight"); plt.close()
print(f"Observation (Model): saved {SENS_TORNADO_PNG.name}")

# Monte Carlo (uncertainty in K, r, start delay)
def monte_carlo_voda(n=2000, K_mean=K, K_sd_rel=0.10, r_mean=r, r_sd=0.12, delay_mu=2, delay_sd=2):
    rng = np.random.default_rng(42)
    Ks = rng.normal(K_mean, K_mean*K_sd_rel, n).clip(min=K_mean*0.7)
    rs = rng.normal(r_mean, r_sd, n).clip(min=0.1)
    delays = rng.normal(delay_mu, delay_sd, n).clip(min=0.0)
    months = np.arange(0, (target_year - start_year)*12 + 1)
    out_units_2030, time_to_1m = [], []
    for Ki, ri, d in zip(Ks, rs, delays):
        t0m = (target_year - start_year)/2.0 * 12.0
        y_m = Ki / (1 + np.exp(-ri*((months - d) - t0m)/12.0))
        out_units_2030.append(y_m[-1])
        idx = np.argmax(y_m >= 1_000_000)
        time_to_1m.append(idx if y_m[idx] >= 1_000_000 else np.nan)
    return np.array(out_units_2030), np.array(time_to_1m)

u2030, t1m = monte_carlo_voda()
pd.DataFrame([{
    "units_2030_p10": float(np.nanpercentile(u2030, 10)),
    "units_2030_p50": float(np.nanpercentile(u2030, 50)),
    "units_2030_p90": float(np.nanpercentile(u2030, 90)),
    "time_to_1m_months_p10": float(np.nanpercentile(t1m, 10)),
    "time_to_1m_months_p50": float(np.nanpercentile(t1m, 50)),
    "time_to_1m_months_p90": float(np.nanpercentile(t1m, 90)),
}]).to_csv(MC_OUT, index=False)
print(f"Observation (Model): wrote {MC_OUT.name}")




Observation (Model): wrote vodafone_suez_sensitivity_one_way.csv
Observation (Model): saved vodafone_suez_sensitivity_units_tornado.png
Observation (Model): wrote vodafone_suez_monte_carlo_summary.csv


### Comparative KPI visuals (Vodafone vs Orange)

In [8]:


kpis = pd.DataFrame({
    "Partnership": ["Vodafone–SUEZ","Orange–Veolia/Birdz"],
    "Target Units (2030)": [K, 3_000_000],
    "Savings %": [int(CONSUMPTION_REDUCTION_PCT*100), np.nan],
    # Vodafone CAPEX modeled; Orange capex/benefit unknown publicly — left NaN (non-synthetic)
    "CAPEX (€m)": [int(CAPEX_PER_METER_EUR*K/1e6), np.nan],
    "Annual Benefit (€m)": [int((K*AVG_ANNUAL_WATER_SPEND_PER_CONNECTION_EUR*CONSUMPTION_REDUCTION_PCT)/1e6), np.nan],
})
kpis["Years to Payback"] = kpis["CAPEX (€m)"] / kpis["Annual Benefit (€m)"]


### ROI bar (only partnerships with both CAPEX & Benefit available)

In [9]:


kpis_for_bar = kpis.dropna(subset=["CAPEX (€m)","Annual Benefit (€m)"])
plt.figure(figsize=(7,5))
plt.bar(kpis_for_bar["Partnership"].astype(str), kpis_for_bar["Years to Payback"])
plt.ylabel("Years"); plt.title("Comparative ROI (payback years) — MODEL (Vodafone only)")
plt.tight_layout(); plt.savefig(ROI_BAR_PNG, bbox_inches="tight"); plt.close()
print(f"Observation: saved {ROI_BAR_PNG.name}")


Observation: saved roi_comparison.png


### Radar chart (normalize numeric KPIs; NaN→0 to visualize scale gaps)

In [10]:

radar = kpis.set_index("Partnership")
radar_norm = radar / radar.max(numeric_only=True)
labels = [c for c in radar_norm.columns if radar_norm[c].notna().any()]
angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False).tolist(); angles += angles[:1]
fig, ax = plt.subplots(figsize=(6,6), subplot_kw=dict(polar=True))
for idx, row in radar_norm.iterrows():
    vals = [0 if pd.isna(row[c]) else row[c] for c in labels]; vals += vals[:1]
    ax.plot(angles, vals, label=idx); ax.fill(angles, vals, alpha=0.25)
ax.set_xticks(angles[:-1]); ax.set_xticklabels(labels, fontsize=9)
ax.set_title("Comparative KPI Radar (normalized)"); ax.legend(loc="upper right", bbox_to_anchor=(1.3, 1.1))
plt.tight_layout(); plt.savefig(RADAR_PNG, bbox_inches="tight"); plt.close()
print(f"Observation: saved {RADAR_PNG.name}")


Observation: saved kpi_radar.png


### Top Countries (allocation MODEL)

In [11]:



def split_countries(s):
    if pd.isna(s): return []
    parts = [p.strip() for p in str(s).replace(",",";").split(";") if p.strip()]
    return parts

rows = []
for _, r_ in league_eu.iterrows():
    countries = split_countries(r_["countries"])
    if not countries or pd.isna(r_["announced_units_or_capacity"]): 
        continue
    share = r_["announced_units_or_capacity"] / len(countries)
    for c in countries:
        rows.append({"country": c, "partnership": r_["partnership"], "allocated_units_MODEL": share})

top_c = pd.DataFrame(rows)
if not top_c.empty:
    top_c_group = top_c.groupby(["country","partnership"]).allocated_units_MODEL.sum().reset_index()
    top5 = top_c_group.sort_values("allocated_units_MODEL", ascending=False).head(5)
    top_c_group.to_csv(TOP_COUNTRIES_CSV, index=False)
    plt.figure(figsize=(8,5))
    for p in top5["partnership"].unique():
        subset = top5[top5["partnership"]==p]
        plt.bar(subset["country"], subset["allocated_units_MODEL"], label=p)
    plt.ylabel("Allocated units (MODEL)"); plt.title("Top Countries by Allocated Target Units (MODEL)")
    plt.legend(); plt.tight_layout(); plt.savefig(TOP_COUNTRIES_PNG, bbox_inches="tight"); plt.close()
    print(f"Observation (Model): saved {TOP_COUNTRIES_PNG.name} and wrote {TOP_COUNTRIES_CSV.name}")
else:
    print("Observation (Model): No country strings to allocate; skipped top-countries outputs.")


Observation (Model): saved top_countries_bar_MODEL.png and wrote top_countries_allocated_MODEL.csv


### Appendices & Executive write-up

In [12]:


# Appendix KPI inputs (with sources) — mirrors what’s used above
pd.DataFrame([
    {"partnership":"Vodafone–SUEZ","sector":"Smart water (NB‑IoT)","target_units_2030":K,
     "savings_pct":int(CONSUMPTION_REDUCTION_PCT*100),"countries":"UK; FR; IT; ES; NZ",
     "start_year":start_year,"target_year":target_year,
     "source_title":"SUEZ × Vodafone press release",
     "source_url":"https://www.suez.com/en/news/press-releases/suez-vodafone-sign-global-partnership-remote-reading-new-generation-smart-water-meters"},
    {"partnership":"Orange–Veolia/Birdz","sector":"Smart water (LoRaWAN)","target_units_2030":3_000_000,
     "savings_pct":np.nan,"countries":"FR","start_year":2018,"target_year":2027,
     "source_title":"Orange newsroom — Birdz 3M meters (LoRa)",
     "source_url":"https://newsroom.orange.com/nova-veolia-and-its-subsidiary-birdz-choose-orange-business-services-to-help-them-digitalize-veolias-remote-water-meter-reading-services-in-france/"}
]).to_csv(APPX_KPI_CSV, index=False)

with open(WRITEUP_MD, "w", encoding="utf-8") as f:
    f.write("# Executive Results & Methodology (Press‑facts only)\n\n")
    f.write("## Data Collection (Open Sources)\n")
    f.write("- SUEZ × Vodafone PR: 2M NB‑IoT smart meters by 2030; up to 15% consumption reduction.\n")
    f.write("- Orange–Veolia/Birdz PR: 3M water meters connected (LoRaWAN) in France.\n")
    f.write("  See appendix CSVs for source titles/URLs.\n\n")
    f.write("## Wrangling & League\n")
    f.write(f"- Consolidated league: `{LEAGUE_OUT.name}` (EU/UK focus). Pace KPI → `{RANKED_OUT.name}`.\n\n")
    f.write("## Predictive Modelling (MODEL)\n")
    f.write(f"- Logistic S‑curve for Vodafone–SUEZ: K={K:,}, start={start_year}, target={target_year}, r={r}; milestones 50%={m50}, 80%={m80}.\n")
    f.write(f"- Projection CSV: `{VODA_PROJ_CSV.name}`; Plot: `{VODA_PROJ_PNG.name}`.\n\n")
    f.write("## Value & ROI (MODEL)\n")
    f.write(f"- Parameters: annual spend €{AVG_ANNUAL_WATER_SPEND_PER_CONNECTION_EUR}, savings {int(CONSUMPTION_REDUCTION_PCT*100)}%, capex/meter €{CAPEX_PER_METER_EUR}.\n")
    f.write(f"- Payback year: {payback_year}; ROI (horizon): {roi_h:.2f}; NPV: €{np.round(npv_est,0):,.0f}; IRR (annual): {('n/a' if np.isnan(irr_a) else str(np.round(irr_a*100,2))+'%')}.\n")
    f.write(f"- Outputs: `{VALUE_CURVE_PNG.name}`, `{ROI_OUT.name}`.\n\n")
    f.write("## Sensitivity & Monte Carlo (MODEL)\n")
    f.write(f"- One‑way sensitivity table: `{SENS_OUT.name}`; tornado: `{SENS_TORNADO_PNG.name}`.\n")
    f.write(f"- Monte Carlo summary: `{MC_OUT.name}` (units by 2030; time to 1M meters).\n\n")
    f.write("## Comparative KPIs & Countries\n")
    f.write(f"- ROI bar (available metrics) → `{ROI_BAR_PNG.name}`; KPI radar → `{RADAR_PNG.name}`.\n")
    f.write(f"- Sector scale chart: `{SECTOR_SMART_WATER_PNG.name}`.\n")
    f.write(f"- Top countries (allocation MODEL): `{TOP_COUNTRIES_CSV.name}`, `{TOP_COUNTRIES_PNG.name}`.\n\n")
    f.write("**Note:** MODEL outputs derive from public endpoints and benchmark parameters; replace with company figures to refine ROI.\n")

print("\n== DONE ==")
print("All CSVs, PNGs, and the write‑up are in:", OUT)


== DONE ==
All CSVs, PNGs, and the write‑up are in: /Users/bella/Downloads/vodafone/Methodology/capstone_outputs_final
